dml


In [0]:
data = [(100, 1), (200, 2), (300, 3), (400, 4), (500, 5)]
df = spark.createDataFrame(data,["cust_id","flight_id"])
display(df)

In [0]:
df.write.format("delta").mode("append").save("/Volumes/deltalake/default/deltavol/dmlsink")

In [0]:
%sql
select * from delta.`/Volumes/deltalake/default/deltavol/dmlsink/`

In [0]:
%sql
update delta.`/Volumes/deltalake/default/deltavol/dmlsink/`
set flight_id = 12
where cust_id = 100

In [0]:
%sql
select * from delta.`/Volumes/deltalake/default/deltavol/dmlsink/`

**deletion vectors 
smart optimization feature used in Delta Lake tables to handle row-level changes—like DELETE, UPDATE, and MERGE—without physically rewriting entire Parquet files.**

In [0]:
#upsert


In [0]:
data = [(100, 10), (200, 2), (300, 30), (400, 40), (900, 9)]
df = spark.createDataFrame(data,["cust_id","flight_id"])

In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, "/Volumes/deltalake/default/deltavol/dmlsink")

# Define the condition for upsert
condition = "target.cust_id = source.cust_id"

# Perform the upsert
delta_table.alias("target").merge(
    df.alias("source"),
    condition
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [0]:
%sql
select * from delta.`/Volumes/deltalake/default/deltavol/dmlsink/`

In [0]:
#schema level changes